## Bidirectional Neural Representation:

In [1]:
# Import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.compat.v1.disable_eager_execution()

2024-09-03 23:33:50.173079: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 23:33:51.090114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##### Bipolar permutation function $f: \{-1,1\}^n \rightarrow \{-1, 1\}^n$
##### Binary permutation function $f : \{0,1\}^n \rightarrow \{0, 1\}^n$

In [2]:
# 3-bit and 4-bit bipolar permutation
def get_data(data_type = "3_bits"):
    """
    Generate the training dataset for a pre-defined 
    3-bit or 4-bit permutation function
    """
    
    if data_type == "3_bits":
        x = np.array([
            [1,1,1], [1,1,-1], [1,-1,1], [1,-1,-1],
            [-1,1,1], [-1,1,-1], [-1,-1,1], [-1,-1,-1]])
        y = x[[6,4,2,0,7,5,3,1]]      
    elif data_type == "4_bits":
        x = np.array([
            [-1,-1,-1,-1],[-1,-1,-1,1],[-1,-1,1,-1],[-1,-1,1,1],
            [-1,1,-1,-1],[-1,1,-1,1],[-1,1,1,-1],[-1,1,1,1],
            [1,-1,-1,-1],[1,-1,-1,1],[1,-1,1,-1],[1,-1,1,1],
            [1,1,-1,-1],[1,1,-1,1],[1,1,1,-1],[1,1,1,1]])
        y = x[[12, 14, 15, 13, 9, 5, 8, 4, 11, 7, 10, 6, 2, 0, 1, 3]]
    else:
        raise ValueError("Wrong input!!! Data type should be 3_bits or 4_bits")
    return x, y

In [13]:
data_type = "3_bits"  # "4_bit"
num_hidden_neurons = 5
mode = 'binary'  # "bipolar"
num_epochs = 5000

In [4]:
X, Y = get_data(data_type)
if mode == 'binary':
  X = 0.5 * (X+1)
  Y = 0.5 * (Y+1)

###### Network Architecture: Input layer <--> Hidden Layer <--> Output Layer

In [5]:
## Define the Placeholders !!!
xf = tf.compat.v1.placeholder(tf.float32, [None, X.shape[1]], name = 'input_forward')
yf = tf.compat.v1.placeholder(tf.float32, [None, Y.shape[1]], name = 'target_forward')

xb = tf.compat.v1.placeholder(tf.float32, [None, X.shape[1]], name = 'input_backward')
yb = tf.compat.v1.placeholder(tf.float32, [None, Y.shape[1]], name = 'target_backward')

In [6]:
## Initialize weight
wgts = {}
wgts['U'] = tf.Variable(
    tf.compat.v1.random_normal(
        [X.shape[1], num_hidden_neurons], 
        stddev = 1.0)
    )
wgts['W'] = tf.Variable(
    tf.compat.v1.random_normal(
        [Y.shape[1], num_hidden_neurons], 
        stddev = 1.0)
    )

In [7]:
# Forward Pass
ah_f = tf.nn.sigmoid(tf.matmul(xf, wgts['U']))
oy_f = tf.matmul(ah_f, tf.transpose(wgts['W']))
ay_f = oy_f

# Backward Pass
ah_b = tf.nn.sigmoid(tf.matmul(yb, wgts['W']))
ox_b = tf.matmul(ah_b, tf.transpose(wgts['U']))
ax_b = ox_b

In [8]:
# Define the loss function and optimizer
with tf.device('/device:CPU:0'):
  if mode == "binary":
    loss_f = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(labels = yf, logits =  ay_f))
    loss_b = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(labels = xb, logits =  ax_b))
  
  elif mode == "bipolar":
    yf_ = 0.5 * (yf + 1.0)
    xb_ = 0.5 * (xb + 1.0)
    loss_f = tf.reduce_mean(
        tf.maximum(2*ay_f, 0)-(2*ay_f*yf_) + tf.math.log(1 + tf.exp(-tf.abs(2*ay_f)))
        )
    loss_b = tf.reduce_mean(
        tf.maximum(2*ax_b, 0)-(2*ax_b*xb_) + tf.math.log(1 + tf.exp(-tf.abs(2*ax_b)))
        )
  loss = loss_f + loss_b
  optimizer = tf.compat.v1.train.GradientDescentOptimizer(
        learning_rate = 0.5).minimize(loss)

In [9]:
val_f = np.array([])
val_b = np.array([])

sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

2024-09-03 23:33:53.589189: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-09-03 23:33:53.595352: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [15]:
# Train the bidirectional network
val_f = np.array([])
val_b = np.array([])

sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

for idx in range(num_epochs):

  # Evaluate the model
  if idx % 50 == 0:
    ef, eb = sess.run([loss_f, loss_b],
                        feed_dict= {xf: X, yf: Y, xb: X, yb: Y})
    val_f = np.append(val_f, ef)
    val_b = np.append(val_b, eb)

  # Update the weights
  sess.run(optimizer, feed_dict= {xf: X, yf: Y, xb: X, yb: Y})

In [16]:
# Evaluate the model
pred_y, pred_x= sess.run([ay_f, ax_b],
                         feed_dict= {xf: X, yf: Y, xb: X, yb: Y})

In [17]:
pred_y = (pred_y >= 0).astype(float)
pred_x = (pred_x >= 0).astype(float)
if mode == "bipolar":
  pred_y = (2*pred_y) - 1
  pred_x = (2*pred_x) - 1

#### Evaluation

In [18]:
# Forward Pass: Compute the approximation error
X - pred_x

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [19]:
# Backward Pass: Compute the approximation error
Y - pred_y

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])